In [64]:
import pandas as pd
import numpy as np
import heapq
import pdb

In [42]:
!ls

ce_reg_pred_0.csv	    iterate.py	     __pycache__
chi_squared.py		    Makefile	     README.md
Datafiles		    myenv	     requirements.txt
datafiles.tar.gz	    node.py	     test.bat
docs			    other_data	     tests
function_names_from_binary  parse.py
guide			    prototype.ipynb


In [43]:
df = pd.read_csv("tests/data-tiniest2/data.txt",delim_whitespace=True)

In [44]:
df.columns

Index(['target', 'num1', 'num2', 'num3'], dtype='object')

In [45]:
df['num1']

0     1.0
1     1.1
2     1.2
3     1.3
4     1.4
5     1.5
6     1.6
7     1.7
8     1.8
9     1.9
10    2.0
11    2.1
12    2.2
13    2.3
14    2.4
15    2.5
16    2.6
17    2.7
18    2.8
19    2.9
20    3.0
21    3.1
22    3.2
23    3.3
24    3.4
25    3.5
26    3.6
27    3.7
28    3.8
29    3.9
Name: num1, dtype: float64

In [46]:
for col in df.columns:
    quartile_edges= np.percentile(df[col].values, [25,50,75,100],method='linear')

In [47]:
quartile_edges

array([0.32841209, 0.55426376, 0.66929288, 0.94757109])

In [48]:
stat_pval = {'num1': 0.3992086966088876, 'num2': 7.5506181559256826e-06, 'num3' : 0.5394739470309722}
 

In [49]:
stat_pval

{'num1': 0.3992086966088876,
 'num2': 7.5506181559256826e-06,
 'num3': 0.5394739470309722}

In [50]:
top_3_keys = {key: value for key, value in stat_pval.items(
        ) if value in heapq.nsmallest(3, stat_pval.values())}

In [51]:
top_3_keys

{'num1': 0.3992086966088876,
 'num2': 7.5506181559256826e-06,
 'num3': 0.5394739470309722}

In [54]:
top_3_keys = sorted(top_3_keys.items(),key=lambda x:x[1])

In [60]:
split_var = top_3_keys[0][0]
col = split_var


In [61]:
x_uniq = df[col].drop_duplicates().sort_values()
cutpoints = x_uniq[:-1] + np.diff(x_uniq)/2


In [62]:
cutpoints

23    0.5
6     1.5
2     2.5
1     3.5
7     4.5
0     5.5
18    6.5
3     7.5
13    8.5
Name: num2, dtype: float64

In [102]:
#for cut in cutpoints:
#    right_idx = df[df[col] < cut].index.values
#    left_idx = df[df[col] >= cut].index.values
#    tot_items = len(right_idx) + len(left_idx)
#    weights = tot_items / tot_items, len(right_idx)/tot_items, len(left_idx)/tot_items
#    # Is this correct calculation of sse using the child node means to get the sse on the left and right?
#    left_resid = df.loc[left_idx, self.tgt] - df.loc[left_idx, self.tgt].mean()
#    right_resid = df.loc[right_idx, self.tgt] - df.loc[right_idx, self.tgt].mean()
#    node_resid = df[self.tgt].values - node.y_mean
#    tot_sse = weights[0]*_sse(node_resid) - weights[1] * \
#        _sse(right_resid) - weights[2]*_sse(left_resid)
#    if smallest_tot_sse == None or smallest_tot_sse > tot_sse:
#        smallest_tot_sse = tot_sse
#        cut_with_smallest_sse = cut

greatest_tot_sse = None
node_sse = ((df.target - df.target.mean())**2).sum()
best_cut = None
for cut in cutpoints:
    right_idx = df[df[col] < cut].index
    left_idx = df.drop(right_idx, axis=0).index
    left_mean = df.loc[left_idx].target.mean()
    right_mean = df.loc[right_idx].target.mean()
    tot_items = len(left_idx) + len(right_idx)
    left_sse = ((df.loc[left_idx].target - left_mean)**2).sum()
    right_sse = ((df.loc[right_idx].target - right_mean)**2).sum()
    weights = 1, len(left_idx) / tot_items, len(right_idx) / tot_items
    cut_sse = weights[0]*node_sse - weights[1]*left_sse - weights[2]*right_sse
    print(cut, cut_sse)
    if greatest_tot_sse == None or cut_sse > greatest_tot_sse:
        greatest_tot_sse = cut_sse
        best_cut = cut
        
        

0.5 235.40700000000004
1.5 582.16
2.5 771.906
3.5 868.299
4.5 882.8450000000001
5.5 843.5490000000001
6.5 754.8296666666668
7.5 639.1356666666667
8.5 314.8893333333333


In [103]:
best_cut

4.5